## DATA VISUALISATION TOOL

1. Stock data visualisation tool 
   
   1. closing price
   2. opening price
   3. volume

##### How the tool works:

    - User will input the number of stocks they want to see 
    - User will then input respective ticker symbols 
    - Tool will generate three interactive plots (stock opening price, stock closing price & stock volume) over a 5 year range

In [61]:
# import libraries 

import yfinance as yf
import pandas as pd
import plotly.express as px
import random 
import plotly.io as pio
from plotly.offline import init_notebook_mode
import warnings
warnings.filterwarnings('ignore')

# fixes for GitHub and Plotly
init_notebook_mode(connected = True)
pio.renderers.default = 'notebook_connected'


In [62]:
# get the dataset and prepare it function

def get_and_prep_dataset(list_of_tickers, call_num):

    # get ticker symbol from list of tickers from user
    set_ticker = list_of_tickers[call_num]

    # get ticker from Yahoo Finance API
    ticker = yf.Ticker(set_ticker)

    # get historical data - 5yr range 
    from_yahoo = ticker.history(period = '5y')

    # set raw_dataset variable 
    raw_dataset = from_yahoo

    # series holding opening prices
    df_op = raw_dataset['Open']

    # series holding closing prices
    df_ser = raw_dataset['Close']
    
    # series holding volume
    df_volume_ser = raw_dataset['Volume']

    # convert series to pandas dataframes 
    df_ = pd.DataFrame(df_op)
    df_.index.name = None # fix for the two rows for column header issue
    df_.reset_index(level = 0, inplace = True)
    df_.rename(columns = {'index' : 'Date'}, inplace = True)

    df = pd.DataFrame(df_ser)
    df.index.name = None 
    df.reset_index(level = 0, inplace = True)
    df.rename(columns = {'index' : 'Date'}, inplace = True)

    df_volume = pd.DataFrame(df_volume_ser)
    df_volume.index.name = None 
    df_volume.reset_index(level = 0, inplace = True)
    df_volume.rename(columns = {'index' : 'Date'}, inplace = True)

    # set name variable from dictionary of company info
    name = ticker.info['longName']

    # set ticker symbol variable from dictionary of company info
    ticker_symbol = ticker.info['symbol']

    return df_, df, df_volume, name, ticker_symbol, ticker


In [63]:
# information function 

def get_company_info(ticker):

    # information for ETFs
    if ticker.info['quoteType'] == 'ETF':
        print(' ')
        print('Stock: '  + ticker.info['longName'])
        print(' ')
        print('//////////////////////////////////////////////////////////////////////////////////////////////////////////////')  
        print(' ')  
        print('Ticker Symbol: ' + ticker.info['symbol'])
        print('Type: Exchange Traded Fund (ETF)')
        print('Fund Family: ' + ticker.info['fundFamily'])
        print('Total Number of Assets: ' + str(ticker.info['totalAssets']))
        print(' ')
        print('Summary: ' + ticker.info['longBusinessSummary'])
        print(' ')
        print('//////////////////////////////////////////////////////////////////////////////////////////////////////////////')   
        print(' ')
        holdings = ticker.info['holdings']
        heading = '{} : {} : {}'.format('SYMBOL', 'HOLDING NAME', 'HOLDING PERCENT')
        print(heading)
        print(' ')
        for i in range(len(holdings)):
            result = '{} : {} : {}'.format(holdings[i]['symbol'], holdings[i]['holdingName'], holdings[i]['holdingPercent'])
            print(result)
        print('\n')

    # information for equities 
    elif ticker.info['quoteType'] == 'EQUITY':  
        print(' ') 
        print('Stock: '  + ticker.info['longName'])
        print(' ')
        print('//////////////////////////////////////////////////////////////////////////////////////////////////////////////')  
        print(' ')
        print('Ticker Symbol: ' + ticker.info['symbol'])
        print('Industry: ' + ticker.info['industry'])
        print('Address: ' + ticker.info['address1'] + ', ' + ticker.info['city'] + ', ' + ticker.info['state'] + ', ' + ticker.info['country'])
        print(' ')
        print('Summary: ' + ticker.info['longBusinessSummary'])
        print(' ')
        print('//////////////////////////////////////////////////////////////////////////////////////////////////////////////')  
        print(' ')
        print('Profit Margin: ' + str(ticker.info['profitMargins']))
        print('Total Revenue: $' + str(ticker.info['totalRevenue']))
        print('Total Cash: $' + str(ticker.info['totalCash']))
        print('Total Debt: $' + str(ticker.info['totalDebt']))
        print('Debt to Equity: ' + str(ticker.info['debtToEquity']))
        print('Operating Cashflow: $' + str(ticker.info['operatingCashflow']))
        print('52 Week Change: ' + str(ticker.info['52WeekChange']))
        print('\n')
    

In [64]:
# visualisation function

def show_visual(df_, df, df_volume, name, ticker_symbol):
    
    # holds all colors that will be used 
    all_colors = ['blue', 'orange', 'red', 'green', 'purple', 'brown', 'violet', 'pink', 'darkgoldenrod', 'black', 'teal', 'gold']

    # visualise stock price at opening
    fig = px.line(df_, x = 'Date', y = 'Open', title = '<b>' + name + ' (' + ticker_symbol + ')' + '</b>' + ' - Stock Price (Open)', labels = {'value' : 'Open Price', 'Open' : 'Open Price ($)'})

    fig.update_layout(height = 575)
    fig.update_traces(hovertemplate = 'Date: %{x} <br>Open Price: $%{y}', line_color = random.choice(all_colors))

    fig.show()

    # visualise stock price at closing
    fig = px.line(df, x = 'Date', y = 'Close', title = '<b>' + name + ' (' + ticker_symbol + ')' + '</b>' + ' - Stock Price (Close)', labels = {'value' : 'Close Price', 'Close' : 'Close Price ($)'})

    fig.update_layout(height = 575)
    fig.update_traces(hovertemplate = 'Date: %{x} <br>Close Price: $%{y}', line_color = random.choice(all_colors))

    fig.show()

    # visualise stock volume at closing
    fig_v = px.area(df_volume, x = 'Date', y = 'Volume', title = '<b>' + name + ' (' + ticker_symbol + ')' + '</b>' + ' - Stock Volume', labels = {'value' : 'Volume', 'Volume' : 'Volume (M)'})

    fig_v.update_layout(height = 575)
    fig_v.update_traces(hovertemplate = 'Date: %{x} <br>Volume: %{y}', line_color = random.choice(all_colors))

    fig_v.show()


In [65]:
# all calls and input requests for the user

# user enters number of stocks to see 
how_many = input('Enter how many stocks: ')

# creates an array that will hold user input for ticker symbols 
list_of_tickers = []
for h in range(int(how_many)):
    input_ticker = input('Enter ticker symbol: ')
    list_of_tickers.append(input_ticker.upper())

# presentation info
print('The tool will be run for ' + str(how_many) + ' stock/s')
print('The stock/s -> ' + str(list_of_tickers))
print('\n')

# function calls 
for call_num in range(len(list_of_tickers)):
    df_, df, df_volume, name, ticker_symbol, ticker = get_and_prep_dataset(list_of_tickers = list_of_tickers, call_num = call_num)
    get_company_info(ticker)
    show_visual(df_, df, df_volume, name, ticker_symbol)


The tool will be run for 4 stock/s
The stock/s -> ['GOOGL', 'BBUS', 'VNQ', 'AAPL']


 
Stock: Alphabet Inc.
 
//////////////////////////////////////////////////////////////////////////////////////////////////////////////
 
Ticker Symbol: GOOGL
Industry: Internet Content & Information
Address: 1600 Amphitheatre Parkway, Mountain View, CA, United States
 
Summary: Alphabet Inc. provides various products and platforms in the United States, Europe, the Middle East, Africa, the Asia-Pacific, Canada, and Latin America. It operates through Google Services, Google Cloud, and Other Bets segments. The Google Services segment offers products and services, including ads, Android, Chrome, hardware, Gmail, Google Drive, Google Maps, Google Photos, Google Play, Search, and YouTube. It is also involved in the sale of apps and in-app purchases and digital content in the Google Play store; and Fitbit wearable devices, Google Nest home products, Pixel phones, and other devices, as well as in the provisio

 
Stock: JPMorgan BetaBuilders U.S. Equity ETF
 
//////////////////////////////////////////////////////////////////////////////////////////////////////////////
 
Ticker Symbol: BBUS
Type: Exchange Traded Fund (ETF)
Fund Family: JPMorgan
Total Number of Assets: 1081726976
 
Summary: The underlying index is a free float adjusted market capitalization weighted index which consists of equity securities primarily traded in the United States. The underlying index targets 85% of those stocks by market capitalization, and primarily includes large- and mid-cap companies. The fund will invest at least 80% of its assets in securities included in the underlying index.
 
//////////////////////////////////////////////////////////////////////////////////////////////////////////////
 
SYMBOL : HOLDING NAME : HOLDING PERCENT
 
AAPL : Apple Inc : 0.0556
MSFT : Microsoft Corp : 0.052800003
AMZN : Amazon.com Inc : 0.0382
FB : Facebook Inc A : 0.0216
GOOGL : Alphabet Inc A : 0.019
GOOG : Alphabet Inc Class

 
Stock: Vanguard Real Estate Index Fund
 
//////////////////////////////////////////////////////////////////////////////////////////////////////////////
 
Ticker Symbol: VNQ
Type: Exchange Traded Fund (ETF)
Fund Family: Vanguard
Total Number of Assets: 82814885888
 
Summary: The advisor attempts to track the index by investing all, or substantially all, of its assets-either directly or indirectly through a wholly owned subsidiary, which is itself a registered investment company-in the stocks that make up the index, holding each stock in approximately the same proportion as its weighting in the index. The fund is non-diversified.
 
//////////////////////////////////////////////////////////////////////////////////////////////////////////////
 
SYMBOL : HOLDING NAME : HOLDING PERCENT
 
VRTPX : Vanguard Real Estate II Index : 0.1153
AMT : American Tower Corp : 0.0749
PLD : Prologis Inc : 0.056199998
CCI : Crown Castle International Corp : 0.0469
EQIX : Equinix Inc : 0.0423
PSA : Public St

 
Stock: Apple Inc.
 
//////////////////////////////////////////////////////////////////////////////////////////////////////////////
 
Ticker Symbol: AAPL
Industry: Consumer Electronics
Address: One Apple Park Way, Cupertino, CA, United States
 
Summary: Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. It also sells various related services. In addition, the company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; AirPods Max, an over-ear wireless headphone; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, HomePod, and iPod touch. Further, it provides AppleCare support services; cloud services store services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts. Additional